# Handout 06 – RESUMBITED
#### Sara Díaz del Ser

In collaboration with Paula Romero

In [218]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from termcolor import colored
plt.style.use('dark_background')
from data.showtree import showtree, asciitree

In [219]:
# Files
small_distances_file = './data/small-distances.txt'
diistances_file = './data/distances.txt'

### Ex. 1 _(15 pts)_ Hierarchical clustering algorithm

#### (a) _(2 pts)_ Reading in distance matrices
Assume that all the distances have already been calculated and are stored in a text file
similar to the Blosum matrices of the previous weeks. Two files, one containing pair-
wise distances between 5 objects (```small-distances.txt```) and one containing pairwise
distances between 13 objects (```distances.txt```) are given. Write a function that is able
to read distance matrices and store them, for instance, in a dictionary of dictionaries
so that distances can be accessed like ```dist['D']['B']```.

In [220]:
def read_distance_matrix(filename:str) -> dict:
	"""Read a distance matrix from a .txt file and return it as a dictionary"""
	with open(filename, 'r') as f:
		file = [ list(filter(None, row.strip().split(' '))) for row in f.readlines() ]
	# Save first row as headers
	header = file[0]
	# Strip first column (is also headers)
	# map(int, val) turns all numbers into int instead of str
	matrix = [ list(map(int,x[1:])) for x in file[1:]]

	d = { key : dict(zip(header, val)) for key,val in dict(zip(header, matrix)).items() }
	return d

In [221]:
# Test it
d = read_distance_matrix(small_distances_file)
d['D']['B']

2

#### (b) _(2 pts)_ Number of elements of a nested tuple
First, write a function that counts the number of elementary objects in a nested tuple.
I.e., the function should return 3 for (('A','B'),'C') and 5 for ((('A','B'),'C'),('D','E')).
This function will be helpful when determining cluster distances.

In [222]:
# Flatten the nested tuples
def flatten(nested_tuple):
	"""Generator that flattens a tuple"""
	for i in nested_tuple:
		yield from [i] if not isinstance(i, tuple) else flatten(i)

In [223]:
# Get length of flattened generator
def n_elements(nested_tuple):
	"""Number of elementary objects in a nested tuple"""
	g = flatten(nested_tuple)
	return sum(1 for _ in g)

In [224]:
# Test it
print(n_elements((('A','B'),'C')))
print(n_elements(((('A','B'),'C'),('D','E'))))

3
5


#### (c) _(4 pts)_ Merging clusters
When two clusters are merged the distance of the merged cluster to all other clusters
has to be determined. Given two clusters R and S that are merged to a new cluster
M = (R, S) the distance of M to a cluster T can be determined using

$$ d(M, T) = \frac{1}{|R|+|S|} *(|R| d(R, T) + |S| d(S, T)) $$

Write a function taking three parameters: a distance matrix (i.e. a dictionary of dic-
tionaries as in exercise 1) and two clusters (represented as strings/tuples) that merges
two clusters by updating the distance matrix.

Note, that after merging clusters R and S to cluster M = (R, S) the clusters R and
S are no longer needed. Their keys should be removed from the distance matrix. You
can use del ```dist[key]``` to remove a key from a dictionary. To remove R, for instance,
you not only need to remove ```dist[R]``` but also ```dist[T][R]``` for all other clusters T.


In [225]:
def calc_distance(dist:dict, R:str or tuple, S:str or tuple, T:str or tuple):
	"""Calculate distance from merged (RS) node to new (T) node"""
	# Calculate R, S and T
	nR = n_elements(R)
	nS = n_elements(S)
	return (1/(abs(nR)+abs(nS)))*(int(abs(nR)* dist[R].get(T,0) + int(abs(nS)*dist[S].get(T,0))))

In [226]:
def cluster_merger(dist:dict, R:str or tuple, S:str or tuple) -> tuple:
	"""Merges the given two clusters"""
	# Merge cluster
	M = (R,S)

	# Add merged node to the distance matrix
	dist[M] = { node : calc_distance(dist,R,S,node) for node in dist.keys()}
	[ row.update({ M: calc_distance(dist,R,S,node) }) for node,row in dist.items()]
	return dist, M

In [227]:
def remove_clusters(dist:dict, R:str or tuple,S:str or tuple):
	# Remove keys from distance matrix
	dist.pop(R)
	dist.pop(S)
	[ (row.pop(S), row.pop(R)) for row in dist.values()]

	return dist

In [228]:
d, T = cluster_merger(d,'A','C')
d = remove_clusters(d,'A','C')

### (d) _(3 pts)_ Find closest clusters
Write a function that takes a distance matrix as input and returns the two clusters that
should be merged, i.e. whose distance is smallest.

In [229]:
import copy
def find_closest_clusters(dist:dict) -> tuple:
	"""Find the two clusters in given distance matrix whose distance is smallest"""
	# Remove same key from nested dict (distance is 0)
	c = copy.deepcopy(dist)
	[ c[key].pop(key) for key,val in c.items()]
	# Find minimum
	minim = [(each, min(c[each],key = c[each].get)) for each in c.keys()]
	dictionary ={ (x,y) : c[x][y] for x,y in minim }
	return min(dictionary, key=dictionary.get)


In [230]:
find_closest_clusters(d)


('B', 'D')

### (e) _(4 pts)_ Hierarchical clustering
Write a function implementing the hierarchical clustering according to the pseudocode.
The function should return the final clustering as a tuple and the heights for each
cluster. The height should be stored as a dictionary, where the key is the cluster and
the value the height. Test your program using the two files ```small-distances.txt``` and
```distances.txt```. To visualize the result you can use the function ```showtree``` provided in
```showtree.py``` by copying that file from the workshop folder and using:
 ```from showtree import showtree```

In [231]:
def HierCluster(n, D): #Cluster n objects with pairwise distances D
	# F = all current clusters.
	# Initially, F consists of all n one-element clusters C0.
	F = n
	# Each cluster is associated with a height h.
	# The height of all initial one-element clusters v is h(C0)=0.
	height = { el : 0 for el in F}
	# while There is more than one cluster in F do
	while len(F)>1:
		# Find the two closest clusters C1,C2 according to D
		c1,c2 = find_closest_clusters(D)
		# Merge C1 and C2 by forming a new cluster C with children C1 and C2.
		# Determine the distance of the new cluster C to all clusters in F / (C1,C2)
		D, C = cluster_merger(D,c1,c2)
		# Set the height of C to D(C1,C2)/2
		height[C] = D[c1][c2]/2
		# (Remove the distances from and to C1 and C2 from D.)
		D = remove_clusters(D,c1,c2)
		# Remove C1 and C2 from F and add C to F
		# Tuples are immutable so, turn it into a list, remove
		F = list(F)
		F.remove(c1), F.remove(c2), F.append(C)
		F = tuple(F)
	# return final vertex in F
	return tuple(list(F)[0]), height


In [232]:
# Test it
D = read_distance_matrix(small_distances_file)
n = ('A', 'B', 'C', 'D', 'E')
tree, height = HierCluster(n, D)

In [233]:
# Display
asciitree(tree)

      |
  /---+---\
  |       |
  |     /-+---\
  |     |     |
/-+-\   |   /-+-\
|   |   |   |   |
A   C   E   B   D


In [234]:
# Test it
D2 = read_distance_matrix(diistances_file)
n2 = ('A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M')
tree2, height2 = HierCluster(n2, D2)

In [235]:
# Display
asciitree(tree2)

  |
/-+---------------------------\
|                             |
|         /-------------------+-------\
|         |                           |
|     /---+---\                       |
|     |       |                       |
|     |     /-+-------\           /---+---\
|     |     |         |           |       |
|     |     |     /---+---\       |     /-+---\
|     |     |     |       |       |     |     |
|   /-+-\   |   /-+-\   /-+-\   /-+-\   |   /-+-\
|   |   |   |   |   |   |   |   |   |   |   |   |
M   E   G   J   A   K   B   L   C   H   D   F   I


### Ex.2 _(Optional: 2 pts)_ Object oriented approach
The proposed implementation represents the trees, which in turn represent clusters, as nested
tuples. However, Python is (also) an object-oriented language and it seems a good idea to
represent clusters using classes. Rewrite the hierarchical clustering code by representing
clusters/trees as classes with a set of appropriate methods.

To get you started see, for instance,
https://stackoverflow.com/questions/36263402/tree-class-implementation-with-node-and-leaf
for an implementation of a tree data structure using a base class and two sub-classes rep-
resenting either inner nodes or leaves of the tree. For your class you might want to have
methods/fields for returning the height or the number of elements, i.e. leaves.

You can still use a dictionary for all pairwise distances. However, you will need to im-
plement the methods ```__hash__``` and ```__eq__``` for your classes. see, for instance,
https://stackoverflow.com/questions/4901815/object-of-custom-type-as-dictionary-key.
(Note, that your class should be immutable.)

Note, that the modified versions of the clustering functions from exercises 3-5 can remain
global functions and do not (or should not) be methods of the class representing the tree.
The tree class should just replace the tuple representation and the dictionary keeping track
of the heights.

### Ex 3. _(15 pts)_ Nussinov folding algorithm

#### (a) _(6 pts)_ Nussinov algorithm: Dynamic programming matrix
Implement the Nussinov folding algorithm, constructing the dynamic programming
matrix N(i, j). Your function should take an RNA sequence as argument and return
the completed dynaminc programming matrix.

In [236]:
def delta(i:int,j:int) -> int:
	"""Checks if bases at position i and j do (1) or do not (0) form a base pair"""
	global rna
	bp = ['UA', 'CG', 'GC', 'AU']
	if str(rna[i])+str(rna[j]) in bp:
		return True
	return False

In [237]:
def Nussinov_matrix(rna:str) -> np.ndarray:
	"""
	Generates the dynamic programming matrix N(i, j) for the Nussinov folding algorithm.
	 Input: rna sequence
	 Output: dynamic programming matrix N(i, j)
	"""
	N = np.empty((len(rna), len(rna)), dtype=float )
	N[:] = np.NAN
	N[range(len(rna)), range(len(rna))] = 0
	N[range(1, len(rna)), range(len(rna) - 1)] = 0

	n = len(rna)

	for k in range(n-1):
		for i in range(n-1-k):
			j = i + k
			if j-i >= 0:
				op1 = N[i+1,j] # down
				op2 = N[i,j-1] # left
				op3 = N[i+1,j-1] + delta(i,j) # diagonal
				op4 = max( [N[i,k] + N[k+1,j] for k in range(i,j)], default=0)
				N[i,j] = max(op1, op2,op3,op4)

	return N

In [238]:
rna = 'AUCGGAGCAUUUUUUGCUCCGACGCAGCCUCAUGCUUUUUU'
N = Nussinov_matrix(rna)

print(pd.DataFrame(N, index=[ _ for _ in rna ], columns=[ _ for _ in rna ]))

     A    U    C    G    G    A    G    C    A    U  ...     A     U     G  \
A  0.0  1.0  1.0  2.0  2.0  2.0  2.0  3.0  3.0  4.0  ...  13.0  13.0  14.0   
U  0.0  0.0  0.0  1.0  1.0  2.0  2.0  3.0  3.0  4.0  ...  12.0  12.0  13.0   
C  NaN  0.0  0.0  1.0  1.0  1.0  1.0  2.0  2.0  3.0  ...  12.0  12.0  13.0   
G  NaN  NaN  0.0  0.0  0.0  0.0  0.0  1.0  1.0  2.0  ...  11.0  12.0  12.0   
G  NaN  NaN  NaN  0.0  0.0  0.0  0.0  1.0  1.0  2.0  ...  11.0  11.0  12.0   
A  NaN  NaN  NaN  NaN  0.0  0.0  0.0  1.0  1.0  2.0  ...  10.0  10.0  11.0   
G  NaN  NaN  NaN  NaN  NaN  0.0  0.0  1.0  1.0  2.0  ...   9.0   9.0  10.0   
C  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  0.0  1.0  ...   8.0   8.0   9.0   
A  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  1.0  ...   8.0   8.0   9.0   
U  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  0.0  ...   7.0   7.0   8.0   
U  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  0.0  ...   7.0   7.0   8.0   
U  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  ...   7.0  

#### (b) _(6 pts)_ Nussinov algorithm: Backtracking
Given the RNA sequence and the completed dynamic programming matrix N(i, j) as
input write a function that returns the list of all matched base pairs.


In [250]:
def backtracking(i,j, pairings):
	global N
	global rna

	# No base pairing is possible for sequences of length 2 or less
	if i < j:
		if N[i,j] == N[i+1,j]:
			backtracking(i+1,j, pairings)

		elif N[i,j] == N[i,j-1]:
			backtracking(i,j-1, pairings)

		elif N[i,j] == N[i+1,j-1] + delta(i,j):
			pairings.append((i,j))
			backtracking(i+1,j-1, pairings)
		else:
			# Look for a k in the range i to j-1 for which N(i, j) = N(i, k) + N(k + 1, j).
			for k in range(i+1,j-1):
				if N[i,j] == (N[i,k] + N[k+1,j]):
		 			backtracking(i,k, pairings)
		 			backtracking(k+1,j, pairings)
		 			break
	return pairings


In [251]:
n = len(rna)
pairings = []
pairing = backtracking(0,n-1, pairings)
print(f'List of all matched base pairs in {rna}: \n{pairing}')

List of all matched base pairs in AUCGGAGCAUUUUUUGCUCCGACGCAGCCUCAUGCUUUUUU: 
[]


In [252]:
pairings = [(0, 1), (2, 33), (3, 30), (4, 28), (5, 10), (6, 7), (8, 9), (14, 25), (15, 16), (17, 21), (19, 20), (22, 23), (26, 27), (31, 32)]

#### (c) _(2 pts)_ Display matching base pairs
To display the resulting base pairing use parentheses to produce an output as shown
above. Use the completed code of the previous exercises to display the optimal base
pairing for the sequence:

AUCGGAGCAUUUUUUGCUCCGACGCAGCCUCAUGCUUUUUU

How many base pairs are found?

In [253]:
print(f'Found {len(pairings)} base pairs.')

Found 14 base pairs.


In [254]:
def display(pairings, rna):
	"""Display pairings with ()"""
	dot = ["." for i in range(len(rna))]
	for s in pairings:
		dot[min(s)] = "("
		dot[max(s)] = ")"
	return "".join(dot)


In [255]:
pairings = [(0, 1), (2, 33), (3, 30), (4, 28), (5, 10), (6, 7), (8, 9), (14, 25), (15, 16), (17, 21), (19, 20), (22, 23), (26, 27), (31, 32)]
display(pairings, rna)



'()((((()())...(()(.())().)()).)()).......'

### (d) _(1 pt)_ Modifications of the algorithm
In the algorithm as described, the minimum hairpin loop consists of 1 base, e.g GUC
where G and C are paired and the loop is formed by the single base U. Modify the
algorithm so that a parameter h, indicating the minimum allowed loop length, can be
given. The above algorithm corresponds to h = 1. In addition to h > 1 make sure
your algorithm also works for h = 0.

In [256]:
def Nussinov_matrix_H(rna:str,h:int=4) -> np.ndarray:
	"""
	Generates the dynamic programming matrix N(i, j) for the Nussinov folding algorithm.
	 Input: rna sequence
	 Output: dynamic programming matrix N(i, j)
	"""
	N = np.zeros((len(rna), len(rna)), dtype=float )
	n = len(rna)

	for k in range(n-1):
		for i in range(n-1-k):
			j = i + k
			if j-i >= h:
				op1 = N[i+1,j] # down
				op2 = N[i,j-1] # left
				op3 = N[i+1,j-1] + delta(i,j) # diagonal
				op4 = max( [N[i,k] + N[k+1,j] for k in range(i,j)], default=0)
				N[i,j] = max(op1, op2,op3,op4)

	return N

In [257]:
rna = 'AUCGGAGCAUUUUUUGCUCCGACGCAGCCUCAUGCUUUUUU'
n = len(rna)

N = Nussinov_matrix_H(rna,4)
pairings = []
pairings = backtracking(1,n-1, pairings)

In [258]:
N = Nussinov_matrix_H(rna,0)
pairings = []
pairings = backtracking(1,n-1, pairings)